In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from IPython.display import HTML
import os

def solve_vec(method, f, y0, rng, h):
    t0, tn = rng
    t = np.arange(t0,tn,h)
    y = np.zeros((len(t),)+y0.shape, np.float64)
    y[0] = y0
    
    for ti in range(1, len(t)):
        y[ti] = method(f, t[ti-1], y[ti-1], h)
    return t, y

def euler_method(f, t, y, h):
    return y + h * f(t, y)

def midpoint_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    return y + h * K2

def RK4_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    K3 = f(t+h*0.5, y+(h*0.5*K2))
    K4 = f(t+h, y+(h*K3))
    return y + h*(K1/6+K2/3+K3/3+K4/6)

In [2]:
def render_frame(y, yprev, l, scale, canvas):
    theta = y[0,0]
    x = y[1,0]
    
    shape = canvas.shape
    pos_x = (shape[0]//2+int(scale*x), shape[0]//2)
    pos_p = (pos_x[0]+int(scale*l*np.sin(theta)), pos_x[1]+int(scale*l*np.cos(theta)))

    pos_p_prev = (int(shape[0]/2+scale*yprev[1,0]+scale*l*np.sin(yprev[0,0])), int(shape[0]/2+scale*l*np.cos(yprev[0,0])))
    cv2.line(canvas, pos_p, pos_p_prev, [0,128,128], 1)
    img = canvas.copy()

    cv2.rectangle(img, (pos_x[0]-8,pos_x[1]-3), (pos_x[0]+8,pos_x[1]+3), [235,240,245], -1)

    radius = 12
    cv2.line(img, pos_x, pos_p, [64,64,64], 2)    
    cv2.circle(img, pos_p, radius, [82,16,213], -1)
    cv2.circle(img, (pos_p[0]+radius//3,pos_p[1]-radius//3), radius//3, [235,240,245], -1)
    return img

def balls_animation(t, y, l, size, pre=''):
    path = 'output/'+pre+'_'+str(time.time())+'.avi'
    if(int(cv2.__version__.split('.')[0]) < 3):
        fourcc = cv2.cv.CV_FOURCC(*'MJPG')
    else:
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    video = cv2.VideoWriter(path, fourcc, len(t)//t[-1], (size,size))
    canvas = np.ones((size,size,3))*[32,40,48]
    for i in range(len(t)):
        ycurr = y[i]
        if(i>0):
            yprev = y[i-1]
        else:
            yprev = ycurr
        frame = render_frame(ycurr, yprev, l, size/4, canvas)
        video.write(frame.astype(np.uint8))
    video.release()
    
    path_mp4 = path[:-4]+'.mp4'
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{path_mp4}'".format(input = path, path_mp4=path_mp4))
    os.remove(path)
    return path_mp4

## Zadanie 5. Wahadło zawieszone na ruchomym bloczku

$ \theta'' = \frac{sin\theta (cos\theta m_2L\theta'^2 + gm_1+gm_2)}{L(m_2cos^2\theta-m_1-m_2)} $

$ x'' = \frac{m_2sin\theta (L\theta'^2+cos\theta g)}{m_2cos^2\theta-m_1-m_2} $

In [3]:
g = 9.80665
L = 1
k = 50
m1 = 1
m2 = 1

r = 0.5
alpha = 1
def f(t, y):
    theta = y[0]
    x = y[1]
    return np.array([
            [theta[1], (np.sin(theta[0])*(np.cos(theta[0])*m2*L*theta[1]**2+g*m1+g*m2))/(L*(m2*np.cos(theta[0])**2-m1-m2)) -r*(theta[1]*L)**alpha],
            [x[1], (m2*np.sin(theta[0])*(L*theta[1]**2+np.cos(theta[0])*g))/(m2*np.cos(theta[0])**2-m1-m2) -r*x[1]**alpha]
        ])

# Initial state
y0 = np.array([[3.14, 0], [0,0]], np.float64)

rng = (0,10)
h = 1/30.

t, y = solve_vec(RK4_method, f, y0, rng, h)
path = balls_animation(t, y, L, 800, pre='t5')

HTML('<video width="800" height="800" controls><source src="'+path+'" type="video/mp4"></video>')